In [12]:
import os
import random
from matplotlib import axis
import scipy.io as sio
import numpy as np
import math
import mne
import nolds
import joblib
import torch
from torch.utils.data import TensorDataset
from sklearn.model_selection import KFold, train_test_split
import model as dl  # Ensure this module contains necessary utility functions
import logging
# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [13]:
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 使用 GPU
seed = 18
dl.seed_everything(seed)

# EEG data parameters
duration = 1250


In [15]:
def importAndCropData(file_paths, duration, labels, data_type='preprocessed'):
    EEG_list = []
    label = []
    srate = None
    for i, file in enumerate(file_paths):
        try:
            if data_type == 'raw':
                raw = mne.io.read_raw_edf(file, preload=True, encoding='latin1',verbose='Warning')
                raw.filter(l_freq=None, h_freq=70)
                data = raw.get_data()[0:19]
                channels = raw.info.get('nchan')
                srate = raw.info.get('sfreq')
            else:
                raw = sio.loadmat(file, uint16_codec='latin1')
                data = raw.get('data')[0:19]
                channels = data.shape[0]
                srate = raw.get('srate_function', [None])[0]
                if srate is None:
                    raise ValueError(f"Sampling rate not found in {file}")

            if data.shape[1] > duration:
                epochs = data.shape[1] // duration
                data_crop = data[:,0:epochs*duration]
            else:
                continue
            label += [labels[i]] * epochs
            channels = 19
            data_new = data_crop.reshape(channels, -1, duration).transpose(1, 0, 2)
            EEG_list.append(data_new)

            logging.info(f"Processed file {file}: {epochs} epochs")
        except Exception as e:
            logging.error(f"Error processing file {file}: {e}")
            continue

    if not EEG_list:
        raise ValueError("No data was loaded. Please check the file paths and formats.")
    
    EEG = np.concatenate(EEG_list)
    label = np.array(label)
    logging.info(f"Total epochs: {EEG.shape[0]}, Normal: {np.sum(label == 1)}, "
            f"MCI: {np.sum(label == 0)}")
    return EEG,label,srate

In [16]:
import os
import warnings
# 忽略 RuntimeWarning 警告
warnings.filterwarnings("ignore", category=RuntimeWarning)
# 定义文件夹路径
base_dir = '糖尿病认知障碍与对照脑电数据'
normal_dir = os.path.join(base_dir, '认知正常')
impaired_dir = os.path.join(base_dir, '认知障碍')

# 获取所有的文件路径
normal_files = [os.path.join(normal_dir, f) for f in os.listdir(normal_dir) if f.endswith('.edf')]
impaired_files = [os.path.join(impaired_dir, f) for f in os.listdir(impaired_dir) if f.endswith('.edf')]

all_files = normal_files + impaired_files
label_single = np.concatenate([np.ones(len(normal_files)), np.zeros(len(impaired_files))],axis=0)
# 将 all_files 和 label_single 中的元素按相同顺序打乱
combined = list(zip(all_files, label_single))
random.shuffle(combined)
all_files[:], label_single[:] = zip(*combined)
original_data,labels,srate = importAndCropData(all_files, duration, label_single, data_type='raw')

Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 70 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 70.00 Hz
- Upper transition bandwidth: 17.50 Hz (-6 dB cutoff frequency: 78.75 Hz)
- Filter length: 95 samples (0.190 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s
2024-11-07 00:37:57,267 - INFO - Processed file 糖尿病认知障碍与对照脑电数据/认知正常/王淑艳.edf: 238 epochs


Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 70 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 70.00 Hz
- Upper transition bandwidth: 17.50 Hz (-6 dB cutoff frequency: 78.75 Hz)
- Filter length: 95 samples (0.190 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s
2024-11-07 00:37:57,961 - INFO - Processed file 糖尿病认知障碍与对照脑电数据/认知正常/余洪涛.edf: 244 epochs


Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 70 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 70.00 Hz
- Upper transition bandwidth: 17.50 Hz (-6 dB cutoff frequency: 78.75 Hz)
- Filter length: 95 samples (0.190 s)



...

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
2024-11-07 00:38:27,604 - INFO - Processed file 糖尿病认知障碍与对照脑电数据/认知障碍/郑俊乔.edf: 215 epochs


Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 70 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 70.00 Hz
- Upper transition bandwidth: 17.50 Hz (-6 dB cutoff frequency: 78.75 Hz)
- Filter length: 95 samples (0.190 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
2024-11-07 00:38:28,029 - INFO - Processed file 糖尿病认知障碍与对照脑电数据/认知正常/龚守利.edf: 194 epochs


Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 70 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 70.00 Hz
- Upper transition bandwidth: 17.50 Hz (-6 dB cutoff frequency: 78.75 Hz)
- Filter length: 95 samples (0.190 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
2024-11-07 00:38:28,562 - INFO - Processed file 糖尿病认知障碍与对照脑电数据/认知正常/孟庆珊.edf: 240 epochs
2024-11-07 00:38:29,118 - INFO - Total epochs: 11310, Normal: 5112, MCI: 6198


In [ ]:
import numpy as np
import nolds

def multiscale_entropy(data, scales=20):
    """
    计算多尺度熵，将长度为 2500 的时间序列转化为 20 个熵值。
    """
    mse_values = []
    for tau in range(1, scales + 1):
        # 生成当前尺度下的降采样序列
        coarse_grained_series = np.array([np.mean(data[i:i + tau]) for i in range(0, len(data) - tau + 1, tau)])
        
        # 计算该尺度的样本熵
        if len(coarse_grained_series) > 2:  # 样本熵要求序列长度足够长
            mse_value = nolds.sampen(coarse_grained_series)
            mse_values.append(mse_value)
        else:
            mse_values.append(np.nan)  # 如果降采样序列太短，记录 NaN
    return mse_values
compressed_data = np.zeros((5643, 43, 20))

# 对每个样本和每个通道计算多尺度熵
for sample in range(5643):
    for channel in range(43):
        # 提取单个通道的时间序列
        time_series = original_data[sample, channel, :]
        # 计算多尺度熵并存储到新的矩阵中
        compressed_data[sample, channel, :] = multiscale_entropy(time_series, scales=20)
        print(f"计算完成第 {sample} 个样本的第 {channel} 个通道")

print("计算完成的多尺度熵数据形状：", compressed_data.shape)


In [ ]:
import torch
import numpy as np
import nolds

def multiscale_entropy(data, scales=20):
    """
    计算多尺度熵，将长度为 2500 的时间序列转化为 20 个熵值。
    """
    mse_values = []
    for tau in range(1, scales + 1):
        # 使用 torch 的张量操作进行降采样
        coarse_grained_series = torch.stack([data[i:i + tau].mean() for i in range(0, len(data) - tau + 1, tau)])
        
        # 将结果转换为 numpy 以计算样本熵
        coarse_grained_series = coarse_grained_series.cpu().numpy()  # 确保数据在 CPU 上
        if len(coarse_grained_series) > 2:
            mse_value = nolds.sampen(coarse_grained_series)
            mse_values.append(mse_value)
        else:
            mse_values.append(np.nan)
    return mse_values

# 将数据转换为 torch tensor 并将其移到 GPU 上
original_data = torch.tensor(original_data, dtype=torch.float32).to("cuda")
compressed_data = torch.zeros((5643, 43, 20), device="cuda")

# 对每个样本和每个通道并行计算多尺度熵
for sample in range(5643):
    for channel in range(43):
        # 提取单个通道的时间序列
        time_series = original_data[sample, channel, :]
        # 将时间序列传入多尺度熵函数
        mse_values = multiscale_entropy(time_series, scales=20)
        # 将结果存储在新张量中
        compressed_data[sample, channel, :] = torch.tensor(mse_values, device="cuda")
        print(f"计算完成第 {sample} 个样本的第 {channel} 个通道")

# 将结果转回 CPU 并转换为 numpy 格式
compressed_data = compressed_data.cpu().numpy()
print("计算完成的多尺度熵数据形状：", compressed_data.shape)


In [17]:
train_indices, test_indices = dl.Split_Sets(10, original_data)

# Ensure output directories exist
ensure_dir("EEGData/70hz/TrainData")
ensure_dir("EEGData/70hz/ValidData")
ensure_dir("EEGData/70hz/TestData")

for fold in range(10):
    try:
        # Split into training and test sets
        train_idx = train_indices[fold]
        test_idx = test_indices[fold]

        train_data = original_data[train_idx,:, : ] 
        train_labels = labels[train_idx]
        test_data = original_data[test_idx,:, : ]
        test_labels = labels[test_idx]
        
        # Further split training data into train and validation sets
        train_data_split, valid_data_split, train_labels_split, valid_labels_split = train_test_split(
            train_data, train_labels, test_size=0.1, random_state=seed, stratify=train_labels
        )
        print(train_data_split.shape,train_labels_split.shape,valid_data_split.shape,valid_labels_split.shape)
        # Convert to PyTorch tensors
        train_tensor = torch.from_numpy(train_data_split).float() # (samples, channels, duration)
        train_labels_tensor = torch.from_numpy(train_labels_split).long()

        valid_tensor = torch.from_numpy(valid_data_split).float()
        valid_labels_tensor = torch.from_numpy(valid_labels_split).long()

        test_tensor = torch.from_numpy(test_data).float()
        test_labels_tensor = torch.from_numpy(test_labels).long()

        # Create TensorDatasets
        train_dataset = TensorDataset(train_tensor, train_labels_tensor)
        valid_dataset = TensorDataset(valid_tensor, valid_labels_tensor)
        test_dataset = TensorDataset(test_tensor, test_labels_tensor)

        # Save datasets
        torch.save(train_dataset, f"EEGData/70hz/TrainData/train_data_{fold + 1}_fold_with_seed_{seed}.pth")
        torch.save(valid_dataset, f"EEGData/70hz/ValidData/valid_data_{fold + 1}_fold_with_seed_{seed}.pth")
        torch.save(test_dataset, f"EEGData/70hz/TestData/test_data_{fold + 1}_fold_with_seed_{seed}.pth")

        logging.info(f"Fold {fold + 1} data saved successfully.")
    except Exception as e:
        logging.error(f"Error processing fold {fold + 1}: {e}")


(9161, 19, 1250) (9161,) (1018, 19, 1250) (1018,)


2024-11-07 00:39:05,787 - INFO - Fold 1 data saved successfully.


(9161, 19, 1250) (9161,) (1018, 19, 1250) (1018,)


2024-11-07 00:39:07,785 - INFO - Fold 2 data saved successfully.


(9161, 19, 1250) (9161,) (1018, 19, 1250) (1018,)


...

2024-11-07 00:39:19,627 - INFO - Fold 8 data saved successfully.


(9161, 19, 1250) (9161,) (1018, 19, 1250) (1018,)


2024-11-07 00:39:21,700 - INFO - Fold 9 data saved successfully.


(9161, 19, 1250) (9161,) (1018, 19, 1250) (1018,)


2024-11-07 00:39:23,812 - INFO - Fold 10 data saved successfully.


In [ ]:
total_folds = 10
train_indices, test_indices = dl.Split_Sets(total_folds, EEG_crop)
# Ensure output directories exist
ensure_dir("EEG_Augemnted_Data/TrainData")
ensure_dir("EEG_Augemnted_Data/ValidData")
ensure_dir("EEG_Augemnted_Data/TestData")
kf = KFold(n_splits=10)
seed = 34  # 设定随机种子

for fold, (train_index, test_index) in enumerate(kf.split(SampleEn_EEG)):
    train_data, test_data = SampleEn_EEG[train_index], SampleEn_EEG[test_index]
    train_labels, test_labels = labels[train_index], labels[test_index]
    
    # 进一步划分训练集和验证集
    train_data_split, valid_data_split, train_labels_split, valid_labels_split = train_test_split(
        train_data, train_labels, test_size=0.1, random_state=seed, stratify=train_labels
    )
    # print(train_data_split.shape,train_labels_split.shape,valid_data_split.shape,valid_labels_split.shape)

    # 转换为 PyTorch 张量
    train_data_split = torch.tensor(train_data_split, dtype=torch.float32)
    valid_data_split = torch.tensor(valid_data_split, dtype=torch.float32)
    train_labels_split = torch.tensor(train_labels_split, dtype=torch.long)
    valid_labels_split = torch.tensor(valid_labels_split, dtype=torch.long)

    test_data = torch.tensor(test_data, dtype=torch.float32)
    test_labels = torch.tensor(test_labels, dtype=torch.long)

    # 创建TensorDatasets
    train_dataset = TensorDataset(train_data_split, train_labels_split)
    valid_dataset = TensorDataset(valid_data_split, valid_labels_split)
    test_dataset = TensorDataset(test_data, test_labels)

    # 保存数据和标签
    torch.save(train_dataset, f"EEG_Augemnted_Data/TrainData/train_data_{fold + 1}_fold_with_seed_{seed}.pth")
    torch.save(valid_dataset, f"EEG_Augemnted_Data/ValidData/valid_data_{fold + 1}_fold_with_seed_{seed}.pth")
    torch.save(test_dataset, f"EEG_Augemnted_Data/TestData/test_data_{fold + 1}_fold_with_seed_{seed}.pth.pth")
    logging.info(f"Fold {fold + 1} data saved successfully.")
